In [1]:
import pandas as pd
import re
import json

# Read the symptom data from the text file
with open(r'C:\Users\prave\Downloads\Symptom_data.txt', encoding='utf-8') as f:
    symptom_data = f.read()

# Read the JSON file to identify the columns required in the output
with open(r'C:\Users\prave\Downloads\Fever.json', encoding='utf-8') as f:
    fever_json = json.load(f)

# Extracting the required keys from the fever_json
fever_keys = ['Symptom'] + [item['text'] for item in fever_json['options']]
print("fever columns:",fever_keys)

# Defining a function to extract the value of a given key from a symptom string
def extract_value(symptom, key):
    start = symptom.find(key) + len(key)
    end = symptom.find('.', start)
    return symptom[start:end].strip()

# Parse the symptom data and create a list of dictionaries
symptoms = []
for symptom in symptom_data.split('►<b>Fever</b>:'):
    if symptom.strip() == '':
        continue
    symptom_dict = {'Symptom': 'Fever'}
    for key in fever_keys[1:]:
        if key in ['Associated symptoms', 'Cough', 'Abdominal pain']:
            continue
        if key in symptom:
            symptom_dict[key] = extract_value(symptom, key).replace('-', '').strip()
    if 'Cough' in symptom:
        cough_value = extract_value(symptom, 'Cough -') if 'Cough -' in symptom else 'Patient denies'
        symptom_dict['Associated symptoms - Cough'] = cough_value.split(',')[0].strip()
    if 'Abdominal pain' in symptom:
        abdominal_pain_value = extract_value(symptom, 'Abdominal pain -') if 'Abdominal pain -' in symptom else 'Patient denies'
        symptom_dict['Associated symptoms - Abdominal pain'] = abdominal_pain_value.split(',')[0].strip()
    symptoms.append(symptom_dict)

# Create a Pandas DataFrame from the list of dictionaries
df = pd.DataFrame(symptoms)

# Add a new 'Sr.No' column
df.insert(0, 'Sr.No', range(1, 1 + len(df)))

# Write the DataFrame to an Excel file
df.to_excel('Output.xlsx', index=False)

fever columns: ['Symptom', 'Duration', 'Nature of fever', 'Timing', 'Severity', 'Associated symptoms']


In [2]:
df

,Sr.No,Symptom,Duration,Nature of fever,Timing,Severity,Associated symptoms - Cough,Associated symptoms - Abdominal pain
0,1,Fever,1 Weeks,Irregular (comes & goes),Evening,Low,Patient denies,Middle
1,2,Fever,1 Days,All day/ Constant,All day,High,Dry,Patient denies
2,3,Fever,3 Days,"All day/ Constant, Irregular (comes & goes), E...",Evening,Moderate,Wet,Left
